# Tłumaczenia maszynowe

W dzisiejszym konkursie Waszym zadaniem będzie implementacja metody opisanej w pracy "Naural machine translation by jointly learning to align and translate". Otrzymacie punkty za implementację i trening modelu, policzenie odpowiednich metryk oraz przeprowadzenie dodatkowych eksperymentów i wizualizacji. W czasie pracy powinniście zadbać o jasność i czytelność kodu, a także estetykę prezentowanych wykresów. Wasza implementacja będzie oceniana pod kątem zgodności z publikacją.

## Zasady konkursu

Musicie przestrzegać poniższych zasad:

- Nie wolno korzystać z Internetu. Wyjątki to OpenAI API, dokumentacja Pytorcha, a także Google Classroom Olimpiady.
- Nie wolno korzystać z Copilota, ani żadnych innych modeli pomagających pisać kod, poza modelami z rodziny GPT3.5.
- Nie wolno korzystać z własnych notatek: zarówno odręcznych, jak i plików na komputerze (wliczając w to w szczególności kod pobrany na komputer).
- Nie wolno łączyć się z zasobami obliczeniowymi innymi niż Google Colab z T4 GPU.

## Zadanie i punktacja

Modele będziesz trenować na zbiorze danych zawierającym pary zdań w języku angielskim i niemieckim (*parallel corpus*). Pamiętajcie o dobrych praktykach pisania kodu i poprawnym formatowaniu, będzie to wpływało na ocenę. Na podstawie załączonej pracy wykonaj następujące podzadania.

**Podzadanie 1: Implementacja modelu (7 p.)**

W tej sekcji prosimy o bardzo dokładne zaimplementowanie metody z pracy.

**Podzadanie 2: Trening modelu (3 p.)**

Wytrenuj model na dostarczonym zbiorze danych (w startowym kodzie są już zaimplementowane dataloadery). Podczas treningu zadbaj o monitorowanie zarówno lossu treningowego jak i walidacyjnego. Następnie sporządź wykresy tych lossów w zależności od iteracji. Wykonaj ewaluację wytrenowanego modelu na podzbiorze testowym dostarczonego zbioru używając właściwych metryk. Możesz skorzystać z metryk wykorzystanych w pracy. Zaprezentuj przykłady zarówno poprawnych jak i niepoprawnych tłumaczeń.

**Podzadanie 3: Wizualizacja atencji (1 p.)**

Zaprezentuj wizualizacje wyuczonych map atencji na przykładzie ciekawych zdań. Możesz wzorować się na wykresach z pracy.

**Podzadanie 4: Dodatkowe eksperymenty (2 p.)**

Jeśli masz pomysły na dodatkowe interesujące eksperymenty, możesz dostać za nie dodatkowe punkty. Możesz rozważyć modyfikacje modelu, ablacje i inne.

## Uwagi
* Możesz zmieniać sygnatury funkcji i klas, a także zaproponowaną przez nas poniżej strukturę kodu. Pamiętaj jednak o dobrych praktykach.

## Kod startowy

In [ ]:
! pip install datasets
! pip install evaluate
! pip install torchtext

# Download the embedding models
! python -m spacy download en_core_web_sm
! python -m spacy download de_core_news_sm

In [2]:
import random

import datasets
import evaluate
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import spacy
import torch
import torch.nn as nn

import torchtext; torchtext.disable_torchtext_deprecation_warning()
import torchtext.vocab; torchtext.disable_torchtext_deprecation_warning()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## Datasety

Przygotowaliśmy dla Ciebie dataloadery oraz tokenizatory.

In [5]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
dataset = datasets.load_dataset("bentrevett/multi30k")

en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

sos_token = "<sos>"
eos_token = "<eos>"


def tokenize_example(example, max_length=1000):
    en_tokens = [token.text.lower() for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text.lower() for token in de_nlp.tokenizer(example["de"])][:max_length]

    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}


train_data = dataset["train"].map(tokenize_example)
valid_data = dataset["validation"].map(tokenize_example)
test_data = dataset["test"].map(tokenize_example)


In [6]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [unk_token, pad_token, sos_token, eos_token]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [7]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def numericalize_example(example):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

train_data = train_data.map(numericalize_example)
valid_data = valid_data.map(numericalize_example)
test_data = test_data.map(numericalize_example)

In [8]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type="torch", columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type="torch",
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type="torch",
    columns=format_columns,
    output_all_columns=True,
)

In [9]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
BATCH_SIZE = 128

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

train_data_loader = get_data_loader(train_data, BATCH_SIZE, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, BATCH_SIZE, pad_index)
test_data_loader = get_data_loader(test_data, BATCH_SIZE, pad_index)

## Podzadanie 1: Implementacja modelu

In [10]:
# Hubert Jastrzebski
# V LO Kraków

import torch.nn.functional as F
#import pdb

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout):
        super().__init__()
        self.encoder_hidden_dim = encoder_hidden_dim

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        outputs = outputs[:, :, self.encoder_hidden_dim:]

        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return outputs, hidden

In [12]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.encoder_hidden_dim = encoder_hidden_dim

        self.attn = nn.Linear(2 * encoder_hidden_dim, encoder_hidden_dim)
        self.va = nn.Linear(encoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(0)

        energy = torch.tanh(self.attn(hidden).unsqueeze(0).expand(seq_len, -1, -1) + encoder_outputs)
        attention = F.softmax(self.va(energy).squeeze(), dim=0)
        result = torch.bmm(attention.transpose(0, 1).unsqueeze(1), encoder_outputs.transpose(0, 1)).squeeze()

        return result

In [13]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embedding_dim,
        encoder_hidden_dim,
        decoder_hidden_dim,
        dropout,
        attention,
    ):
        super().__init__()
        self.decoder_hidden_dim = decoder_hidden_dim

        self.attention = attention
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.dropout = nn.Dropout(dropout)

        self.rnn = nn.GRU(encoder_hidden_dim + embedding_dim, decoder_hidden_dim, bidirectional=True)
        self.fc_out = nn.Linear(2 * encoder_hidden_dim + decoder_hidden_dim + embedding_dim, output_dim)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))
        attention = self.attention(hidden, encoder_outputs)

        rnn_input = torch.cat((embedded, attention), dim=1).unsqueeze(0)

        hidden = hidden.unsqueeze(0)
        hidden = torch.cat((hidden[:,:,self.decoder_hidden_dim:], hidden[:,:,:self.decoder_hidden_dim]), dim=0)
        output, hidden = self.rnn(rnn_input, hidden)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)

        output = torch.cat((embedded, attention, output.squeeze()), dim=1)
        prediction = self.fc_out(output)

        return prediction, hidden, attention

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        src = src.to(device)
        trg = trg.to(device)
        encoder_outputs, hidden = self.encoder(src)

        input = trg[0]
        outputs = torch.zeros(trg.shape[0], trg.shape[1], self.decoder.output_dim, device=device)

        for t in range(1, trg.shape[0]):
            prediction, hidden, attention = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = prediction
            input = trg[t]

        return outputs

In [15]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5

attention = Attention(encoder_hidden_dim, decoder_hidden_dim).to(device)
encoder = Encoder(input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout
).to(device)
decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
).to(device)
model = Seq2Seq(encoder, decoder).to(device)

## Podzadanie 2: Trening modelu

In [ ]:
# TO!DO: Napisz pętlę treningową. Pamiętaj o zbieraniu statystyk treningowych i walidacyjnych.

def compute_loss(model, criterion, data_loader):
    total_loss = 0.0
    total_samples = 0

    model.eval()
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)

    average_loss = total_loss / total_samples

    return average_loss

def train_model(
    model,
    opimizer,
    criterion,
    train_loader,
    valid_loader,
    num_epochs,
    verbose=True,
    plot=True
):
    train_losses, valid_losses = [], []
    max_norm = 1.0
    for epoch in range(num_epochs):
        for data in train_loader:
            model.train()
            src, trg = data['en_ids'], data['de_ids']

            optimizer.zero_grad()

            output = model(src, trg)

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            loss.backward()

            optimizer.step()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm)

            train_losses.append(loss.item())

            val_loss = compute_loss(model, valid_loader, criterion)
            valid_losses.append(val_loss)

    return train_losses, valid_losses

# TO!DO: Wytrenuj model na dostarczonym zbiorze danych
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.95, eps=1e-6)
criterion = nn.CrossEntropyLoss()
train_losses, valid_losses = train_model(model, optimizer, criterion, train_data_loader, valid_data_loader, 1)

In [ ]:
# TO!DO: Sporządź wykresy funkcji kosztu

plt.plot(train_losses, label='train')
plt.plot(valid_losses, label='valid')
plt.legend()
plt.show()

In [ ]:
# TO!DO: Zewaluuj model na zbiorze testowym

In [ ]:
def translate_sentence(sentence, model):
    en_tokens = []  # Initialize list for English tokens
    de_tokens = []  # Initialize list for German tokens
    attention = []  # Initialize list for attention scores

    # Tokenize the input sentence
    input_tensor = model.source_vocab.get_ids_from_sentence(sentence)
    input_length = len(input_tensor)
    input_tensor = torch.LongTensor(input_tensor).unsqueeze(1).to(model.device)

    # Pass the input through the model to get the output and attention
    with torch.no_grad():
        output, attn = model(input_tensor)

    # Get the predicted token IDs and attention scores
    output_ids = output.argmax(dim=2)
    attention_scores = attn.squeeze(1)

    for i in range(output_ids.size(0)):
        en_tokens.append(model.target_vocab.get_token(output_ids[i].item()))
        de_tokens.append(model.source_vocab.get_token(input_tensor[i].item()))
        attention.append(attention_scores[i].tolist())

    return en_tokens, de_tokens, attention


# TO!DO: Przykłady tłumaczeń

## Podzadanie 3: Wizualizacja atencji

In [ ]:
def plot_attention(sentence, translation, attention):
    pass
    # TO!DO


# TO!DO: Wizualizacja atencji

## Podzadanie 4: Dodatkowe eksperymenty


In [ ]:
# TO!DO